<a href="https://colab.research.google.com/github/zzzle96/PyTorch/blob/master/convlstm%20basis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies for video module in pytorch to work
! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libavformat-dev is already the newest version (7:3.4.8-0ubuntu0.2).
libavformat-dev set to manually installed.
The following additional packages will be installed:
  libavfilter-dev libpostproc-dev
The following NEW packages will be installed:
  libavdevice-dev libavfilter-dev libpostproc-dev
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,154 kB of archives.
After this operation, 5,444 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libpostproc-dev amd64 7:3.4.8-0ubuntu0.2 [51.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavfilter-dev amd64 7:3.4.8-0ubuntu0.2 [1,016 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavdevice-dev amd64 7:3.4.8-0ubuntu0.2 [87.2 kB]
Fetched 1,154 kB in 1s (799 kB/s)
Selecting previously unselected package libpost

In [2]:
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

--2020-09-20 05:59:05--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2020-09-20 05:59:05--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  27.5MB/s    in 59s     

2020-09-20 06:00:04 (34.4 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]

--2020-09-20 06:00:04--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
R

In [4]:
# Import required modules ...

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
#import av
import random
#print(f"PyAV version -- {av.__version__}")

SEED = 491
torch.manual_seed(SEED)

from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

#run_av_diagnostics()

In [5]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar 


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open test_train_splits.rar
No such file or directory
No files to extract
rm: cannot remove 'test_train_splits.rar': No such file or directory

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open hmdb51_org.rar
No such file or directory
No files to extract
rm: cannot remove 'hmdb51_org.rar': No such file or directory
mv: cannot stat '*.rar': No such file or directory


In [6]:
# diagnostics for PyAV installation. For now version 6.2.0 works 
def run_av_diagnostics():
    import av
    av.open("video_data/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.avi")
    print(get_video_backend())
    av.logging.set_level(av.logging.ERROR)
    if not hasattr(av.video.frame.VideoFrame, 'pict_type'):
      print("Nah")

run_av_diagnostics()

pyav


In [7]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=ac17c4bfa22e8792a65f739927b2e3ee36eaa191a652d078d86113abc0d5c534
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  |     Proc size: 282.2 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total     15079MB


In [8]:
# download video transformation functions from below link
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
import transforms as T

--2020-09-20 06:02:24--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py’

transforms.py       100%[===================>]   3.03K  --.-KB/s    in 0s      

2020-09-20 06:02:25 (47.1 MB/s) - ‘transforms.py’ saved [3102/3102]



In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvLSTM(nn.Module):
    def __init__(self, n_in, output_channel, class_num, in_ch):
        super(ConvLSTM, self).__init__()
        self.in_ch = in_ch
        self.n_in   = n_in
        self.out_ch = output_channel 
        self.rnn1 = Conv_LSTM(input_channel = self.in_ch, output_channel = self.out_ch)
        self.ap = nn.AvgPool1d(32,32, padding=0)
        self.fcl1 = nn.Linear(self.out_ch * self.n_in // 32, class_num, bias = False)
        
                
    def forward(self, x, hidden):
        h1 = hidden
        x = x.reshape(-1, self.in_ch, self.n_in)
        x, h1 = self.rnn1(x, h1)
        x = F.relu(x)
        x = self.ap(x)
        x = x.view(-1, self.out_ch * self.n_in // 32)
        x = self.fcl1(x)
        hidden = h1
                
        return x, hidden

class Conv_LSTM(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(Conv_LSTM, self).__init__()
        self.output_channel = output_channel
        self.fcl_wxf = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxi = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxo = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxg = nn.Conv1d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whf = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whi = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_who = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whg = nn.Conv1d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)

    def forward(self, input, hidden):
        h_t_1, c_t_1 = hidden
        f_t_i = (self.fcl_wxf(input) + self.fcl_whf(h_t_1))
        i_t_i = (self.fcl_wxi(input) + self.fcl_whi(h_t_1))
        g_t_i = (self.fcl_wxg(input) + self.fcl_whg(h_t_1))
        o_t_i = (self.fcl_wxo(input) + self.fcl_who(h_t_1))

        f_t = torch.sigmoid(f_t_i)
        i_t = torch.sigmoid(i_t_i)
        g_t = torch.tanh(g_t_i)
        o_t = torch.sigmoid(o_t_i)
                
        c_t = f_t * c_t_1 + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        output = h_t
        hidden = (h_t, c_t)
        
        return output, hidden
        
    def initHidden(self, batch_size, seg_size):
        weight = next(self.parameters())
        return (weight.new_zeros(batch_size, self.output_channel, seg_size),
                weight.new_zeros(batch_size, self.output_channel, seg_size))  